In [ ]:
pip install 'fsspec>=0.3.3'

In [ ]:
pip install modin

In [ ]:
pip install modin[dask]

In [ ]:
pip install ray

In [ ]:
pip install datatable

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Data Glacier internship/week6')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np 
import pandas as pd
import zipfile
import sqlite3
import time
import dask.dataframe as dd
import modin.pandas as modin_pd
import datatable as dt
import gzip


###Take any csv/text file of 2+ GB of your choice.

###Read the file ( Present approach of reading the file)

###Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational efficiency

In [7]:
%%time

text_df = pd.read_csv('eng.csv', index_col=[0])

CPU times: user 41.2 s, sys: 5.41 s, total: 46.7 s
Wall time: 52 s


In [ ]:
%%time

text_dask_df = dd.read_csv("eng.csv", sample=1000000, dtype='object', engine='python', on_bad_lines='skip').compute()

CPU times: user 1min 5s, sys: 5.84 s, total: 1min 11s
Wall time: 1min 10s


In [ ]:
%%time
from distributed import Client
client = Client()

text_modin_df = modin_pd.read_csv("eng.csv")

(deploy_ray_func pid=1428) tcmalloc: large alloc 1073741824 bytes == 0x83b8e000 @  0x7f73f40ba2a4 0x7f71f317e9a5 0x7f71f317fcc1 0x7f71f318169e 0x7f71f315250c 0x7f71f315f399 0x7f71f314797a 0x59afff 0x515655 0x549e0e 0x593fce 0x511e2c 0x549576 0x593fce 0x511e2c 0x593dd7 0x5118f8 0x549576 0x4bca8a 0x5134a6 0x549e0e 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x4bc98a 0x7f73f0d92e02
(deploy_ray_func pid=1430) tcmalloc: large alloc 1073741824 bytes == 0x83c56000 @  0x7f72e2a822a4 0x7f70e1b459a5 0x7f70e1b46cc1 0x7f70e1b4869e 0x7f70e1b1950c 0x7f70e1b26399 0x7f70e1b0e97a 0x59afff 0x515655 0x549e0e 0x593fce 0x511e2c 0x549576 0x593fce 0x511e2c 0x593dd7 0x5118f8 0x549576 0x4bca8a 0x5134a6 0x549e0e 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x4bc98a 0x7f72df75ae02
(deploy_ray_func pid=1429) tcmalloc: large alloc 1073741824 bytes == 0x846c4000 @  0x7f8c920a12a4 0x7f8a9115f9a5 0x7f8a91160cc1 0x7f8a9116269e 0x7f8a9113350c 0x7f8a91140399 0x7f8a9

CPU times: user 7.63 s, sys: 3.55 s, total: 11.2 s
Wall time: 53.7 s


In [ ]:
%%time

data_dtable = dt.fread("eng.csv", fill=True)
data_dtable = data_dtable.to_pandas()

CPU times: user 32.3 s, sys: 33.3 s, total: 1min 5s
Wall time: 21.3 s


Pandas - 52 sec

Dask - 1 min 10 sec

Modin - 53 sec

Datatable - 21 sec

### Perform basic validation on data columns : eg: remove special character , white spaces from the col name



In [9]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


### As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of read and write file, column name in YAML

In [8]:
%%writefile file.yaml
file_type: csv
dataset_name: eng_text
file_name: eng
table_name: text_info
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - title
    - url
    - abstract
    - body_text
    - body_html

Writing file.yaml


### Validate number of columns and column name of ingested file with YAML.

In [4]:
import testutility as util
config_data = util.read_config_file("file.yaml")

In [5]:
file_type = config_data['file_type']
source_file = config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'], index_col=[0])
df.head()

,title,url,abstract,body_text,body_html
0,Wikibooks: Radiation Oncology/NHL/CLL-SLL,https://en.wikibooks.org/wiki/Radiation_Oncolo...,Chronic Lymphocytic Leukemia and Small Lymphoc...,Front Page: Radiation Oncology | RTOG Trials |...,"<div class=""mw-parser-output""><table width=""10..."
1,Wikibooks: Romanian/Lesson 9,https://en.wikibooks.org/wiki/Romanian/Lesson_9,==Băuturi/Beverages==,Băuturi/Beverages[edit | edit source]\nTea : C...,"<div class=""mw-parser-output""><h2><span id=""B...."
2,Wikibooks: Karrigell,https://en.wikibooks.org/wiki/Karrigell,Karrigell is an open Source Python web framewo...,Karrigell is an open Source Python web framewo...,"<div class=""mw-parser-output""><p>Karrigell is ..."
3,Wikibooks: The Pyrogenesis Engine/0 A.D./GuiSe...,https://en.wikibooks.org/wiki/The_Pyrogenesis_...,====setupUnitPanel====,setupUnitPanel[edit | edit source]\nHelper fun...,"<div class=""mw-parser-output""><h4><span class=..."
4,Wikibooks: LMIs in Control/pages/Exterior Coni...,https://en.wikibooks.org/wiki/LMIs_in_Control/...,== The Concept ==,Contents\n\n1 The Concept\n2 The System\n3 The...,"<div class=""mw-parser-output""><div id=""toc"" cl..."


In [7]:
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [8]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['title', 'url', 'abstract', 'body_text', 'body_html'], dtype='object')
columns of YAML are: ['title', 'url', 'abstract', 'body_text', 'body_html']


### Write the file in pipe separated text file (|) in gz format.

In [9]:
df.to_csv('eng_pipe',sep = '|')

In [13]:
pipe_file = open("eng_pipe","rb").read()
bin_data = bytearray(pipe_file)
with gzip.open("eng_pipe.gzip","wb") as f:
    f.write(bin_data)

In [3]:
with gzip.open("eng_pipe.gzip", "rb") as f:
    data = f.read()

In [8]:
data[:1000]

b'|title|url|abstract|body_text|body_html\n0|Wikibooks: Radiation Oncology/NHL/CLL-SLL|https://en.wikibooks.org/wiki/Radiation_Oncology/NHL/CLL-SLL|Chronic Lymphocytic Leukemia and Small Lymphocytic Lymphoma (CLL/SLL)|"Front Page: Radiation Oncology | RTOG Trials | Randomized Trials\n\n\n\n\nNon-Hodgkin lymphoma: Main Page  | Randomized\nOverview: Overview  | \nFollicular |\nDiffuse large B-cell |\nMALT |\nNodal marginal zone |\nMantle cell |\nCLL/SLL |\nLymphoblastic |\nBurkitt |\nNK/T cell |\nAnaplastic large cell |\nPrimary CNS Lymphoma\nTreatment:\nAggressive |\nSpecific sites |\nRadioimmunotherapy\n\n\n\nChronic Lymphocytic Leukemia and Small Lymphocytic Lymphoma (CLL/SLL)\n\n\nContents\n\n1 Overview\n2 Staging\n3 Classification\n4 Richter\'s transformation\n5 Radiation Therapy\n6 Reviews\n\n\n\nOverview[edit\xc2\xa0| edit source]\nCLL is the most common leukemia among adults in Western world\nIt is characterized by accumulation of mature B-cells\nCLL molecular phenotype: CD5+ CD2

### Create a summary of the file:

Total number of rows,

total number of columns

file size

In [21]:
text_df.shape #86736 rows, 5 columns

(86736, 5)

In [20]:
import os
b = os.path.getsize("eng.csv")
print(f'The file size is {round(b/1024/1024/1024, 2)} GB')

The file size is 3.01 GB
